In [0]:
!wget http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
!wget http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz
!gunzip train-images-idx3-ubyte.gz
!gunzip train-labels-idx1-ubyte.gz
!ls -l
!pip install idx2numpy

In [0]:
import numpy as np
import idx2numpy as idx
import matplotlib.pyplot as plt
import random
import torch.nn as nn
import torch.utils as utils
import torch

In [0]:
train_x=idx.convert_from_file("train-images-idx3-ubyte")
train_y=idx.convert_from_file("train-labels-idx1-ubyte")
train_x=train_x.reshape(60000,28,28,1)
train_x=train_x.transpose((0,3,1,2))
train_x=train_x.astype(np.float32)
train_y=train_y.astype(np.int64)
train_x=train_x/255

In [0]:
from sklearn.model_selection import train_test_split
features_train, features_test, targets_train, targets_test = train_test_split(train_x,
                                                                             train_y,
                                                                             test_size = 0.2,
                                                                             random_state = 42)

In [0]:
batch_size=128
features_train = torch.from_numpy(features_train)
targets_train = torch.from_numpy(targets_train)
features_test = torch.from_numpy(features_test)
targets_test = torch.from_numpy(targets_test)
train = utils.data.TensorDataset(features_train,targets_train)
test = utils.data.TensorDataset(features_test,targets_test)
train_loader = utils.data.DataLoader(train, batch_size = batch_size, shuffle = True)
test_loader = utils.data.DataLoader(test, batch_size = len(features_test), shuffle = False)

In [0]:
class Conv_Model(nn.Module):
    def __init__(self):
        super(Conv_Model,self).__init__()
        self.conv_1=nn.Conv2d(in_channels=1,out_channels=32,kernel_size=3,stride=1)
        self.conv_2=nn.Conv2d(in_channels=32,out_channels=64,kernel_size=5,stride=1)
        self.mpool=nn.MaxPool2d(2)
        self.relu=nn.ReLU()
        self.flat=nn.Flatten()
        self.dense_1=nn.Linear(64*4*4,120)
        self.dense_2=nn.Linear(120,10)

    def forward(self,x):
        x=self.relu(self.conv_1(x))
        x=self.mpool(x)
        x=self.relu(self.conv_2(x))
        x=self.mpool(x)
        x=self.flat(x)
        x=self.dense_1(x)
        x=self.dense_2(x)
        return x


In [0]:
Model=Conv_Model()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
Model.to(device)
loss=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(Model.parameters(),lr=0.00001)
num_epochs=20

for j in range(num_epochs):
    for i,(images,labels) in enumerate(train_loader):
        images=torch.autograd.Variable(images).to(device)
        labels=torch.autograd.Variable(labels).to(device)

        output=Model(images)
        l=loss(output,labels)
        l.backward()
        optimizer.step()
    for i,(images,labels) in enumerate(test_loader):
        images=torch.autograd.Variable(images).to(device)
        labels=torch.autograd.Variable(labels).to(device)
        output=Model(images)
        l=loss(output,labels)
        output=torch.max(output,1)[1]
        acc=(output==labels).sum()
        acc=float(acc)/len(labels)
        print("Epoch:{} Acc:{} Loss:{}".format(j+1,acc,l.data))

Epoch:1 Acc:0.8031666666666667 Loss:1.4650548696517944
Epoch:2 Acc:0.8558333333333333 Loss:0.5332047343254089
Epoch:3 Acc:0.8765833333333334 Loss:0.4326862096786499
Epoch:4 Acc:0.8798333333333334 Loss:0.4906684160232544
Epoch:5 Acc:0.8804166666666666 Loss:0.533950686454773
Epoch:6 Acc:0.9111666666666667 Loss:0.3739830255508423
Epoch:7 Acc:0.9193333333333333 Loss:0.31411394476890564
Epoch:8 Acc:0.92275 Loss:0.2670436501502991
Epoch:9 Acc:0.9355833333333333 Loss:0.23408503830432892
Epoch:10 Acc:0.9463333333333334 Loss:0.18040411174297333
Epoch:11 Acc:0.9551666666666667 Loss:0.1661156415939331
Epoch:12 Acc:0.9590833333333333 Loss:0.15378491580486298
Epoch:13 Acc:0.9615 Loss:0.1496596783399582
Epoch:14 Acc:0.9638333333333333 Loss:0.14104284346103668
Epoch:15 Acc:0.9641666666666666 Loss:0.13157355785369873
Epoch:16 Acc:0.9673333333333334 Loss:0.1183348074555397
Epoch:17 Acc:0.9685 Loss:0.11245077848434448
Epoch:18 Acc:0.969 Loss:0.10539961606264114
Epoch:19 Acc:0.97225 Loss:0.10156736522912

In [0]:
plt.imshow(train_x[4000].transpose(1,2,0).reshape(28,28))
plt.show()

In [0]:
epsilon=0.0001
def fgsm(data,image):
    data_grad_sign=data.sign()
    n_image=image+epsilon*data_grad_sign
    n_image=torch.clamp(n_image,0,1)
    return n_image

In [0]:
perturbed_images=[]
for k in range(5000):
    sample=random.randrange(60000)
    image_data=train_x[sample]
    target_original=train_y[sample]
    image_data_true=train_x[sample].reshape(28,28)
    while(1):
        target=random.randrange(10)
        if target!=train_y[sample]:
            target=[target]
            break
    for i in range(100):
        data=torch.autograd.Variable(torch.tensor(image_data.reshape(1,1,28,28))).to(device)
        data.requires_grad=True
        label=torch.autograd.Variable(torch.tensor(np.array(target))).to(device)
        output=Model(data)
        l=loss(output,label)
        Model.zero_grad()
        l.backward()
        new_image=fgsm(data.grad.data,data)
        temp=new_image.data.cpu().numpy().reshape(28,28)
        if output.max(1)[1].cpu().data.numpy()[0]!=target_original:
            print(output.max(1)[1])
            plt.figure(figsize=(1,1))
            plt.imshow(temp,cmap="gray",vmin=0,vmax=1)
            plt.show()
            print(np.sum(np.abs(image_data_true-temp)))
            perturbed_images.append({"image":temp,"original_label":target_original})
            break
        image_data=temp
